In [6]:
!pip install "matplotlib>=3.2,<=3.7.3"
!pip install "ydata-profiling>4.4,<4.5"
!pip install tensorflow

DEPRECATION: fairness 0.1.8 has a non-standard dependency specifier BlackBoxAuditing>=0.1.26ggplot. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of fairness or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
^C
ERROR: Operation cancelled by user
DEPRECATION: fairness 0.1.8 has a non-standard dependency specifier BlackBoxAuditing>=0.1.26ggplot. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of fairness or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [7]:
import pandas as pd

from functions import *
import numpy as np
import operator

#np.random.seed(1)
#X_train, X_test, y_train, y_test, sensitive, sensitivet = DATA_TRAIN_TEST(1,'sex',"income-per-year",['sex','income-per-year','race-sex'])  #,'race','race-sex','sex'

In [8]:
class DatasetWithForcedDistribution:
    def __init__(self, sensitive_attribute_name, distribution, X_train, X_test, y_train, y_test, sensitive, sensitive_t):
        self.sensitive_attribute_name = sensitive_attribute_name
        self.distribution = distribution
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        self.sensitive = sensitive
        self.sensitive_t = sensitive_t
        

In [9]:
pd.value_counts(sensitive)

1    13603
0     6505
dtype: int64

In [16]:

distributions = [0.1, 0.2]#, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]  # fraction of samples with sensitive=1

all_datasets = []

def droppy(df, idx):
    return df.reset_index().drop(index=idx).drop(columns=["index"])


def find_indices_to_drop(sensitive, target_distribution):
    length = len(sensitive)
    indices_to_drop = []
    def current_dist(sensitive_value=1):
        if sensitive_value == 1:
            return (pd.value_counts(sensitive)[1] - len(indices_to_drop)) / (length - len(indices_to_drop))
        else:
            return (pd.value_counts(sensitive)[1]) / (length - len(indices_to_drop))

    if current_dist() > target_distribution:
        comp = operator.gt
        sensitive_value_to_delete = 1
    else:
        comp = operator.lt
        sensitive_value_to_delete = 0

    i = 0
    while comp(current_dist(sensitive_value_to_delete), target_distribution):
        if sensitive[i]==sensitive_value_to_delete:
            indices_to_drop.append(i)
        i = i + 1
        
    return indices_to_drop

for dist in distributions:
    print(f'now computing dist {dist}')
    np.random.seed(1)
    X_train, X_test, y_train, y_test, sensitive, sensitive_t = DATA_TRAIN_TEST(1,'sex',"income-per-year",['sex','income-per-year','race-sex'])
    
    indices_to_drop = find_indices_to_drop(sensitive, dist)
    indices_to_drop_t = find_indices_to_drop(sensitive_t, dist)

    df = DatasetWithForcedDistribution(
        sensitive_attribute_name = 'sex',
        distribution = dist,
        X_train = droppy(X_train, idx=indices_to_drop),
        X_test = droppy(X_test, idx=indices_to_drop_t),
        y_train = droppy(y_train, idx=indices_to_drop),
        y_test = droppy(y_test, idx=indices_to_drop_t),
        sensitive = np.delete(sensitive, indices_to_drop, 0),
        sensitive_t = np.delete(sensitive_t, indices_to_drop_t, 0)
    )
    all_datasets.append(df)

now computing dist 0.1


/home/js/git/adversarial-bb-training/functions.py:126: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X_train = X_train.drop(columns_delete,1)
/home/js/git/adversarial-bb-training/functions.py:127: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X_test = X_test.drop(columns_delete,1)


now computing dist 0.2


/home/js/git/adversarial-bb-training/functions.py:126: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X_train = X_train.drop(columns_delete,1)
/home/js/git/adversarial-bb-training/functions.py:127: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X_test = X_test.drop(columns_delete,1)


In [ ]:
for ds in all_datasets:
    print(pd.value_counts(ds.sensitive) / len(ds.sensitive))

In [ ]:
ds.y_train

In [17]:
#for ds in all_datasets:
ds = all_datasets[0]

In [28]:

gb = GradientBoostingClassifier(n_estimators=500, learning_rate = 0.05, max_depth = 3, max_features=90,random_state=0,  min_impurity_decrease=0.0,
                                min_samples_leaf=2, min_samples_split=2,
                                min_weight_fraction_leaf=0.0)
gb.fit(ds.X_train, ds.y_train)

/home/js/anaconda3/envs/adversarial-bb-training/lib/python3.10/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier(learning_rate=0.05, max_features=90,
                           min_samples_leaf=2, n_estimators=500,
                           random_state=0)

In [ ]:
y_pred = gb.predict(ds.X_test.values)

In [ ]:
# gb.predict_proba(ds.X_test.values) # !!

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(ds.y_test["income-per-year"], y_pred)

In [ ]:
X_train_pr, _, _, _, _, _ = DATA_TRAIN_TEST(1,'sex',"income-per-year",['sex','income-per-year','race-sex'])

In [ ]:
import ydata_profiling

ydata_profiling.ProfileReport(X_train_pr)

In [20]:
ds.X_train.shape

(7227, 97)

In [30]:
random_length = 300

In [31]:
random_dict = {}
for attr in ds.X_train:
    print(f'{round(X_train_pr[attr].min(),2)} {round(X_train_pr[attr].max(),2)}')
    random_attr = np.random.rand(random_length)*(X_train_pr[attr].max()+abs(X_train_pr[attr].min()))-abs(X_train_pr[attr].min())
    random_dict[attr] = random_attr

-1.63 3.94
-3.58 2.31
-2.47 0.4
-0.15 13.4
-0.22 10.76
-3.34 4.85
-0.18 5.62
-0.27 3.68
-1.68 0.6
-0.19 5.16
-0.3 3.32
-0.21 4.71
-0.02 44.83
-0.17 5.96
-0.19 5.35
-0.11 8.99
-0.07 14.0
-0.1 10.21
-0.14 7.19
-0.12 8.24
-0.18 5.47
-0.22 4.61
-0.45 2.23
-0.11 8.99
-0.7 1.44
-0.24 4.15
-0.04 26.78
-0.14 7.41
-0.53 1.88
-0.4 2.5
-0.03 36.6
-0.94 1.06
-0.11 9.2
-0.69 1.45
-0.18 5.58
-0.17 6.02
-0.37 2.67
-0.02 57.88
-0.39 2.54
-0.39 2.54
-0.18 5.48
-0.22 4.61
-0.26 3.79
-0.34 2.93
-0.07 14.51
-0.39 2.55
-0.15 6.67
-0.37 2.72
-0.18 5.67
-0.24 4.23
-0.84 1.18
-0.59 1.71
-0.18 5.71
-0.42 2.4
-0.34 2.93
-0.22 4.55
-0.03 39.32
-0.06 16.8
-0.05 21.35
-0.04 22.98
-0.06 17.84
-0.05 20.03
-0.03 32.52
-0.06 17.84
-0.05 19.64
-0.03 31.69
-0.06 16.02
-0.03 32.52
-0.05 21.6
-0.04 25.45
-0.01 141.8
-0.02 57.88
-0.02 42.74
-0.02 47.26
-0.06 16.92
-0.04 27.27
-0.03 36.6
-0.05 21.86
-0.05 19.09
-0.05 20.88
-0.03 39.32
-0.14 7.14
-0.03 35.44
-0.02 47.26
-0.03 34.38
-0.08 12.85
-0.04 23.61
-0.03 29.55
-0.06 1

In [32]:
random_ds = pd.DataFrame(random_dict)

In [33]:
random_ds

,age,education-num,race,capital-gain,capital-loss,hours-per-week,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,-0.341585,-0.276075,-0.443886,9.902258,0.997499,4.133365,2.892329,3.280893,-1.609508,2.882419,...,28.324142,1.743609,27.051170,8.419170,17.613443,10.431213,14.077892,-1.650000,10.250436,7.645530
1,-0.834133,-0.319917,-1.308346,4.672400,6.037236,-1.769039,2.401414,1.200259,-1.601420,2.498215,...,29.165701,16.344917,45.187133,18.703565,15.504126,13.253633,21.682932,-2.212371,14.413412,32.550095
2,-0.146089,-0.977029,0.140472,2.419523,10.012216,2.190774,-0.146020,2.181786,-1.119446,4.570097,...,12.325566,10.647561,49.607300,14.598764,9.775153,0.069538,4.552584,-2.290610,19.861420,3.633544
3,1.995140,-2.637425,-1.928773,0.897730,9.896598,-1.720436,1.816193,0.093687,0.206481,0.162663,...,15.892994,16.549307,44.553314,11.766583,10.399065,25.370046,25.591076,-0.036760,1.000753,22.716603
4,1.517468,-0.842447,-2.128235,1.728006,10.030556,3.702781,3.522343,1.791405,-1.474639,3.152261,...,27.320278,14.178426,49.158695,18.940200,14.230635,35.160253,18.864085,-1.945280,18.244618,19.414323
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,-0.478198,0.876770,-0.463016,8.554835,9.401703,4.675726,3.061632,-0.261752,-0.708796,1.249141,...,29.255712,10.732403,27.726490,12.145024,20.481805,28.604276,5.607566,-3.218579,17.566446,38.857085
296,0.357670,-1.966372,-0.177667,10.918937,5.140978,-0.003986,4.513418,3.492415,-1.669956,5.128863,...,23.961204,3.230167,41.456071,19.878805,5.819578,4.377473,28.106531,0.113882,16.678128,3.474693
297,2.667022,-0.623071,-1.792720,2.927198,7.693916,-0.568633,1.184984,0.071276,-0.871344,3.997399,...,1.775460,15.375930,15.341958,20.395002,17.283092,11.240673,9.234206,-2.977754,10.599366,32.691343
298,0.440193,-2.971842,-1.170570,3.376302,-0.096238,1.122017,1.801267,2.105464,-0.276369,4.628767,...,15.634381,11.716053,30.578649,10.716388,9.999653,5.550413,26.228351,-1.223171,1.482520,40.171141


rand*(max+abs(min))+min

In [34]:
gb.predict(random_ds)

array([1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1])